In [1]:
import basilica
import joblib
import numpy as np
import pandas as pd
from flask import jsonify
import sklearn.decomposition
import sklearn.neighbors
import sklearn.preprocessing

In [32]:
class Recommender():
    """
    Recommend cannabis strains for users based on input_text.
    """
    def __init__(self):
        self.scaler = joblib.load('scaler.pkl')
        self.pca = joblib.load('pcaer.pkl')
        self.normalizer = joblib.load('normd.pkl')
        self.nn = joblib.load('nnmodel.pkl')
        self.API_KEY='API_KEY'
        
    def strain_recommender(self, input_text, neighbors=5):
        """
        Uses Basilica to embed input_text from the user and then runs it through a process
        that includes a Standard Scaler, PCA to reduce dimensionality down to 75, 
        normalizes input_text, then a nearest neighbors model that returns 5 recommended
        strains id's for the user.
        """
        with basilica.Connection(self.API_KEY) as c:
            embedded = c.embed_sentence(input_text)
        
        embedded = np.stack([embedded], axis=0)
        
        scaled = self.scaler.transform(embedded)
        pcaed = self.pca.transform(scaled)
        normalized = self.normalizer.transform(pcaed)
        results = self.nn.kneighbors(normalized, neighbors)[1][0].tolist()
        # used this to see distances for recommended strains:
#         score, strain_index = self.nn.kneighbors(normalized, neighbors)
#         print(score, strain_index)
        return results

In [30]:
target = "I need something to help with anxiety and pain but has a sweet flavor"
recommender = Recommender()
recommender.strain_recommender(target)



[188, 1075, 272, 1333, 116]

In [31]:
type(recommender.strain_recommender(target))

list